In [1]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
import preprocess as pr

ModuleNotFoundError: No module named 'preprocess'

In [54]:
from sklearn.model_selection import train_test_split

data = pd.read_csv("../data/data_train.csv", index_col=0)
data_company_response_binary = data[~data["Company response"].isin(["In progress", "Untimely response"])]
estado_respuesta = {'Closed with monetary relief': 0, 
       'Closed with non-monetary relief':0, 'Closed with explanation': 1, 'Closed': 1}

data_company_response_binary["Company response"] = data_company_response_binary["Company response"].map(estado_respuesta)


X = data_company_response_binary.drop(columns=["Company response"])
y = data_company_response_binary["Company response"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 42)

C:\Users\Fernando\AppData\Local\Temp\ipykernel_12264\3351688478.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_company_response_binary["Company response"] = data_company_response_binary["Company response"].map(estado_respuesta)


### Funciones Sub-product



In [31]:
# Filling NaN

filling_na = FunctionTransformer(lambda x: x.fillna("No Subproduct"),validate=False)

# data["Sub-product"] = filling_na.transform(data["Sub-product"])

In [ ]:
# Codificacion con Target Encoder y sustitución de la columna Sub-issue

class FillEncodeColumn(BaseEstimator, TransformerMixin):
    def __init__(self, column, smooth="auto", cv=5, categories = "auto", random_state=42):
        self.column = column
        self.smooth = smooth
        self.cv = cv
        self.random_state = random_state
        self.categories = categories
        self.encoder = None

    def fit(self, X, y=None):
        self.encoder= TargetEncoder(
            smooth=self.smooth,
            cv=self.cv,
            random_state=self.random_state,
            categories = self.categories 
        )
        self.encoder.fit(X[[self.column]], y)
        self.n_features_in_ = X.shape[1]
        return self

    def transform(self, X):
        assert X.shape[1] == self.n_features_in_
        encoded = self.encoder.transform(X[[self.column]])
        X[self.column] = encoded
        return X



In [ ]:
pipeline = Pipeline([
    ("fill_na", filling_na),
    ("encoder", FillEncodeColumn(column="Sub-product"))
])

data = pipeline.fit_transform(X_train, y_train )

## Hasta aqui conseguido

## Funciones Dates

In [ ]:
date_converter = FunctionTransformer(pr.date_converter, validate=False)
day_of_week_month_received = FunctionTransformer(pr.day_of_week_and_month, kw_args={"column_date": "Date received"}, validate = False)
day_of_week_month_sent = FunctionTransformer(pr.day_of_week_and_month, kw_args={"column_date": "Date sent to company"}, validate = False)

# Al contrario que el notebook de preprocessing, he incorporado el drop
# en la función, por lo que elimina la columna original al aplicar la función

## Funciones Product

In [ ]:
class ProductEncoder(BaseEstimator, TransformerMixin):

    def __init__(self,column, sparse_output = False, dtype=int):
        self.column = column
        self.sparse_output = sparse_output
        self.dtype = dtype
        self.encoder = None

    def fit(self, X, y=None):
        self.encoder = OneHotEncoder(sparse_output=self.sparse_output,
                                     dtype=self.dtype)
        self.encoder.fit(X[[self.column]], y)
        self.n_features_in_ = X.shape[1]
        return self
    
    def transform(self, X):
        assert X.shape[1] == self.n_features_in_
        encoded= self.encoder.transform(X[[self.column]])
        encoded = pd.DataFrame(encoded, 
                               columns = self.encoder.get_feature_names_out([self.column]), 
                               index=X.index)
        X = pd.concat([X, encoded], axis=1)
        X = X.drop(columns=[self.column])
        return X
        

## Funciones State

In [ ]:
regions_and_divisions = FunctionTransformer(pr.regions_and_divisions, kw_args={"column_state": "State"}, validate=False)

In [ ]:
class RegionDivisionEncoder(BaseEstimator, TransformerMixin):
    
    def __init__(self,column, sparse_output = False, dtype=int):
        self.column = column
        self.sparse_output = sparse_output
        self.dtype = dtype
        self.encoder = None

    def fit(self, X, y=None):
        self.encoder = OneHotEncoder(sparse_output=self.sparse_output,
                                     dtype=self.dtype)
        self.encoder.fit(X[[self.column]], y)
        self.n_features_in_ = X.shape[1]
        return self
    
    def transform(self, X):
        assert X.shape[1] == self.n_features_in_
        encoded= self.encoder.transform(X[[self.column]])
        encoded = pd.DataFrame(encoded, 
                               columns = self.encoder.get_feature_names_out([self.column]), 
                               index=X.index)
        X = pd.concat([X, encoded], axis=1)
        X = X.drop(columns=[self.column])
        return X
    
    # Recuerda que hay que hacerlo para la columna regions y divisions

## Functions Issue

In [ ]:
# Limpieza de texto

text_cleaner = FunctionTransformer(pr.tokenize_column,kw_args={"column_state": "Issue"}, validate=False)
